Github repo: https://github.com/thewitking/billing-system/


File path: https://github.com/thewitking/billing-system/tree/master/image-dataset-preparation-tools
Contributed by: MANISH SONI

Description: This file is a utility file to convert vgg image annotation (csv) to pascal voc format(xml).
The simplest available free image annotation tool VGG image annotator annotates image in this format which should be converted to any keras-retinanet supported annotation format such as pascal voc format.

I used this utility file to convert vgg image annotations to pascal voc format.
Steps: 
1. Go to http://www.robots.ox.ac.uk/~vgg/software/via/via-1.0.6.html and upload the image dataset file in small batches and use the given tool to highlight the required object.
2. Get the data file (annotation csv file) and change the variables as per the folder structe and as per the product.

In [ ]:
#Github repo: https://github.com/thewitking/billing-system/
#file path: https://github.com/thewitking/billing-system/tree/master/image-dataset-preparation-tools

# file is kindaa messy with unwanted statement that i will correct soon
segmentedvalue=0
# segmented target object should be put in a different folder with its name
foldername='reese'
#path of that product
folderpath='/Users/thewitking/Documents/dataset/'
#csv file downloaded after annotation
annotation_file='vgg_annotations.csv'
#pacal voc demo file 
pacal_ref_file='pascal_voc_ref.xml'

lines=[]
linesxml=[]


#fields
filenames=[]
filename=''
shape=''
xminvals=[]
yminvals=[]
xmaxvals=[]
ymaxvals=[]
xval=0
yval=0
width=0
height=0



    
import csv

with open(annotation_file) as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        shape_row=row[5]
        segs=shape_row.split(',')
        for val in segs:
            if "x" in val:
                partsval=val.split(":")
                xval=partsval[1]
                xminvals.append(xval)
                filename=row[0]
                print("filename=",filename)
                filenames.append(filename)
            if "y" in val:
                partsval=val.split(":")
                yval=partsval[1]
                yminvals.append(yval)
            if "width" in val:
                partsval=val.split(":")
                width=partsval[1]
                xmaxval=int(xval)+int(width)
                xmaxvals.append(str(xmaxval))
            if "height" in val:
                partsval=val.split(":")
                heightstr=partsval[1]
                exactval=heightstr.split("}")
                height=exactval[0]
                ymaxval=int(yval)+int(height)
                ymaxvals.append(str(ymaxval))

from PIL import Image
count=0
recentfile=''
import xml.etree.ElementTree as ET
for file in filenames:
    tree=ET.Element('a')
    if recentfile != file:
        tree = ET.parse(pacal_ref_file)
    else:
        filenameparts=file.split('.')
        xmlfilename=filenameparts[0]+'.xml'
        tree=ET.parse(xmlfilename)
    root = tree.getroot()
    folderval=root.find('folder')
    folderval.text=foldername
    fileval=root.find('filename')
    fileval.text=file
    pathval=root.find('path')
    pathval.text =folderpath+foldername+'/'+file
    
    if recentfile != file:
        for obj in root.iter('object'):
            nameval=obj.find('name')
            nameval.text=foldername
            for bndbox in obj.iter('bndbox'):
                vb= bndbox.find('xmin')
                vb.text = xminvals[count]
                ymin= bndbox.find('ymin')
                ymin.text = yminvals[count]
                xmax= bndbox.find('xmax')
                xmax.text = xmaxvals[count]
                ymaxst= bndbox.find('ymax')
                ymaxst.text = ymaxvals[count]
    
    else:
        obj2= root.find('object')
        newobj =ET.Element('object')
        for child in obj2:
            b=child.tag
            bobj =ET.SubElement(newobj,b)
            if b=='bndbox':
                for bndboxelements in child:
                    cd=bndboxelements.tag
                    cobj =ET.SubElement(bobj,cd)
                    cobj.text=bndboxelements.text
            bobj.text=child.text
            for bndbox in newobj.iter('bndbox'):
                vb= bndbox.find('xmin')
                vb.text = xminvals[count]
                ymin= bndbox.find('ymin')
                ymin.text = yminvals[count]
                xmax= bndbox.find('xmax')
                xmax.text = xmaxvals[count]
                ymaxst= bndbox.find('ymax')
                ymaxst.text = ymaxvals[count]
        root.append(newobj)
        
    im = Image.open(filenames[0])
    wd, hg = im.size
    sizeroot=root.find('size')
    widthval=sizeroot.find('width')
    heightval=sizeroot.find('height')
    widthval.text=str(wd)
    heightval.text=str(hg)
    segmentedval=root.find('segmented')
    segmentedval.text= str(segmentedvalue)
    recentfile=file
    filenameparts=file.split('.')
    xmlfilename=filenameparts[0]+'.xml'
    tree.write(xmlfilename)
    count=count+1
print("success")